In [22]:
import functions_framework
import pandas as pd
import requests 
import sqlalchemy
from datetime import datetime
from pytz import timezone

@functions_framework.http
def update_arrival_flights(request):
    # Establish MySQL database connection string
    connection_string = connection()

    # Fetch airport codes from the 'airports' table
    airports_code = get_airports_code(connection_string)

    # Fetch arrival flight information
    arrival_info = arrival_flights(airports_code)

    # Store arrival flight information in SQL
    arrivals_data_to_sql(arrival_info, connection_string)

    return "Arrival flights information updated successfully."

def connection():
    schema = "gans"
    host = "34.79.246.249"
    user = "root"
    password = "Arsenal2024"
    port = 3306
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    return connection_string

def get_airports_code(connection_string):
    try:
        airports_df = pd.read_sql('airports', con=connection_string)
        return list(airports_df['airport_code'])
    except Exception as e:
        print(f"Error fetching airport codes: {e}")
        return []

def arrival_flights(list_of_airport_codes):
    try:
        today = datetime.now()
        tomorrow = today + timedelta(days=1)
        tomorrow_str = tomorrow.strftime("%Y-%m-%d")
        time_periods = [["00:00", "11:59"], ["12:00", "23:59"]]
        frame = []

        for airport_code in list_of_airport_codes:
            for start_time, end_time in time_periods:
                url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport_code}/{tomorrow_str}T{start_time}/{tomorrow_str}T{end_time}"
                querystring = {"direction": "Arrival", "withCancelled": "false"}
                headers = {
                    "X-RapidAPI-Key": '14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4',
                    "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
                }

                response = requests.get(url, headers=headers, params=querystring)
                response.raise_for_status()

                response_json = response.json()
                if 'arrivals' in response_json:
                    arrivals_df = pd.json_normalize(response_json['arrivals'])
                    arrivals_df['airport_code'] = airport_code
                    frame.append(arrivals_df)
                else:
                    print(f"No arrival data found for airport with code {airport_code}")

        
                arrivals_data = pd.concat(frame, ignore_index=True)
        arrivals_data = arrivals_data[["airport_code", "number", "airline.name", "movement.scheduledTime.local", "movement.terminal", "movement.airport.name", "movement.airport.icao"]]
        arrivals_data.rename(columns = {"number": "flight_nember", "airline.name": "airline_name", "movement.scheduledTime.local": "arrival_time", "movement.terminal": "arrival_terminal", "movement.airport.name": "departure_airport_name", "movement.airport.icao": "departure_airport_code"} , inplace = True)
        arrivals_data['arrival_time'] = pd.to_datetime(arrivals_data['arrival_time'])
      

        return arrivals_data

    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
        return pd.DataFrame()
    except requests.exceptions.RequestException as req_err:
        print(f"Request error occurred: {req_err}")
        return pd.DataFrame()
    except Exception as e:
        print(f"An error occurred: {e}")
        return pd.DataFrame()

def arrivals_data_to_sql(arrivals_data, connection_string):
    try:
        if not arrivals_data.empty:
            arrivals_data.to_sql('flights', if_exists='append', con=connection_string, index=False)
        else:
            print("No data to insert into SQL")
    except Exception as e:
        print(f"Error inserting data into SQL: {e}")

